In [1]:
import os
import datetime

from time import perf_counter
from tqdm import tqdm

from fetchbim import Client, Family
from fetchbim.utils import truncate
from fetchbim.teams import sync_complete_msg
from notion_client import Client as NotionClient

In [2]:
STATUS_DICT = {
    0: "Public", 
    1: "Private", 
    2: "Work in Progress",
    3: "Website Public / Fetch Private"}

In [3]:
def get_all_pages(client, database_id: str, **kwargs) -> list:
    results = []
    cursor = None
    while True:
        response = client.databases.query(
            database_id=database_id, start_cursor=cursor
        )
        results.extend(response["results"])
        if response["has_more"] is False:
            break
        else:
            cursor = response["next_cursor"]
            
    return results

In [4]:
def get_item_in_list(list_, name, default=None):
    for i in list_:
        if i.name.lower() == name.lower():
            try:
                return i.value
            except AttributeError:
                return default
    return default

In [5]:
fetch = Client(auth=os.environ["BIM_KEY"])
notion = NotionClient(auth=os.environ["NOTION_KEY"])

In [7]:
notion.options.timeout_ms = 100000

In [8]:
ifc = get_all_pages(notion, database_id="3288117b879f450cb58b14580fe041ab")
revit_cat = get_all_pages(notion, database_id="3fa7e9f3905f432eaa9c8645b1ce0f98")
bo_cat = get_all_pages(notion, database_id="30ee9042b5cb4ddfa41d6d82c3b7afff")
master = get_all_pages(notion, database_id="a1a8d1982a9b4dc8b8cca008b16ed986")
uniformat = get_all_pages(notion, database_id="09e15c47c13249f1b0a34f3734111151")
omni = get_all_pages(notion, database_id="0effd533b8a74310a8e816802f4b4c4b")

In [9]:
revit_cat_dict = {x['properties']['Category']['title'][0]['plain_text']: x['id'] for x in revit_cat}
bo_cat_dict = {x['properties']['Combined Name']['formula']['string']: x['id'] for x in bo_cat}
ifc_dict = {x['properties']['name']['title'][0]['plain_text']: x['id'] for x in ifc}
master_dict = {x['properties']['code']['rich_text'][0]['plain_text']: x['id'] for x in master}
uniformat_dict = {x['properties']['code']['rich_text'][0]['plain_text']: x['id'] for x in uniformat}
omni_dict = {x['properties']['code']['rich_text'][0]['plain_text']: x['id'] for x in omni}

In [10]:
family_ids = fetch.families.list()

In [11]:
db_id = "f56ac916a3f049dda2df0f864ca63c62"

In [12]:
t1_start = perf_counter()
count = 0
for family_id in tqdm(family_ids):
    filt = {"property": "SSGFID",
        "rich_text": {"equals": family_id}}
    results = notion.databases.query(database_id=db_id, filter=filt)['results']
    if results:
        count += 1
        # family data
        page_id = results[0]['id']
        fam = fetch.families.retrieve(family_id)
        cover = {"type": "external", "external": {"url": f"https://bimservice.ssgbim.com:443/Family/{family_id}/File/FamilyImageLarge"}}
        properties = dict()
        properties['Name'] = {"title": [{"type": "text", "text": {"content": fam.name}}]}
        properties['_Status'] = {"select": {"name": STATUS_DICT.get(fam.status)}}
        documents = "\n".join([f"[{_file.id}] - {_file.key} - {_file.name+_file.extension}" for _file in fam.files if "Gallery" not in _file.key])
        properties['_Documents'] = {"rich_text": [{"text": {"content": truncate(documents)}}]}
        
        # Parameters
        zM = get_item_in_list(fam.parameters, 'zM', default="")
        properties["_zM"] = {"rich_text": [{"text": {"content": zM}}]}
        
        short_descript = get_item_in_list(fam.parameters, 'SSG_Short Description', default="")
        properties["_Short Description"] = {"rich_text": [{"text": {"content": truncate(short_descript)}}]}
        
        long_descript = get_item_in_list(fam.parameters, 'SSG_Long Description', default="")
        properties["_Long Description"] = {"rich_text": [{"text": {"content": truncate(long_descript)}}]}
        
        # Properties
        tags = get_item_in_list(fam.properties, 'Tags', default="")
        if "," in tag_list:
            tags = "\n".join([tag.strip() for tag in tags.split(',')])
        properties["_Tags"] = {"rich_text": [{"text": {"content": truncate(tags)}}]}
        
        detail = get_item_in_list(fam.properties, 'Detail', default="")
        properties["_Detail"] = {"rich_text": [{"text": {"content": truncate(detail)}}]}
        
        tech_data = get_item_in_list(fam.properties, 'Technical Data', default="")
        properties["_Technical Data"] = {"rich_text": [{"text": {"content": truncate(tech_data)}}]}
        
        fam_design = get_item_in_list(fam.properties, 'Family Design', default="")
        properties["_Family Design"] = {"rich_text": [{"text": {"content": truncate(fam_design)}}]}
        
        connectors = get_item_in_list(fam.properties, 'Has MEP Connectors', default=False)
        if connectors == "Yes":
            c_bool = True
        else:
            c_bool = False
        properties["_Has MEP Connectors"] = {"checkbox": c_bool}
        
        pricing = get_item_in_list(fam.properties, 'Includes Pricing', default=False)
        if pricing == 'Yes':
            p_bool = True
        else:
            p_bool = False
        properties["_Includes Pricing"] = {"checkbox": p_bool}
        
        ada = get_item_in_list(fam.properties, 'ADA Compliant', default=False)
        if ada == 'Yes':
            a_bool = True
        else:
            a_bool = False
        properties["_ADA Compliant"] = {"checkbox": a_bool}
        
        sms = get_item_in_list(fam.properties, 'SMS Purchase Link', default="")
        if sms:
            properties['_SMS Purchase Link'] = {"url": sms}
        
        product_id = get_item_in_list(fam.properties, 'product_id', default="")
        if product_id:
            properties['_Product Page'] = {"url": f"https://fetchbim.com/catalog/product/view/id/{str(product_id)}"}
        
        # relations
        bo_cat_value = get_item_in_list(fam.properties, 'BIMobject Category')
        bo_cat_id = bo_cat_dict.get(bo_cat_value)
        if bo_cat_id:
            properties["_BIMobject Category"] = {"relation": [{"id": bo_cat_id}]}
        
        ifc_value = get_item_in_list(fam.properties, 'IFC')
        ifc_id = ifc_dict.get(ifc_value)
        if ifc_id:
            properties["_IFC"] = {"relation": [{"id": ifc_id}]}
            
        omni_value = get_item_in_list(fam.properties, 'Omniclass')
        omni_id = omni_dict.get(omni_value)
        if omni_id:
            properties["_Omniclass"] = {"relation": [{"id": omni_id}]}
            
        uniformat_value = get_item_in_list(fam.parameters, 'Assembly Code')
        uniformat_id = uniformat_dict.get(uniformat_value)
        if uniformat_id:
            properties["_Uniformat"] = {"relation": [{"id": uniformat_id}]}
            
        master_value = get_item_in_list(fam.parameters, 'Keynote')
        master_id = master_dict.get(master_value)
        if master_id:
            properties["_Masterformat"] = {"relation": [{"id": master_id}]}
            
        revit_cat_id = revit_cat_dict.get(fam.category_name.split('/')[0])
        if revit_cat_id:
            properties["_Revit Category"] = {"relation": [{"id": revit_cat_id}]}
            
        response = notion.pages.update(page_id=page_id, properties=properties, cover=cover)
        
t1_stop = perf_counter()
script_time_s = t1_stop - t1_start
script_time = "{:0>8}".format(str(datetime.timedelta(seconds=script_time_s)))
table_dict = dict()
table_dict['Public Ids'] = len(family_ids) 
table_dict['Synced Pages'] = count 
table_dict['Elapsed Time'] = script_time      
sync_complete_msg(
    title="Family Sync Complete",
    message="Notion database 'Content Calendar' has successfully been synced with admin database",
    button=("Content Calendar", "https://www.notion.so/fetchbim/f56ac916a3f049dda2df0f864ca63c62"),
    table_dict=table_dict
)

100%|██████████| 707/707 [22:32<00:00,  1.91s/it]
